how to convert anki package to study sauce?


how to sign a file url for uploading to google storage?


In [ ]:
const {Storage} = require('@google-cloud/storage');
const storage = new Storage();

/**
 * HTTP function that generates a signed URL
 * The signed URL can be used to upload files to Google Cloud Storage (GCS)
 *
 * @param {Object} req Cloud Function request context.
 * @param {Object} res Cloud Function response context.
 */
exports.getSignedUrl = (req, res) => {
  if (req.method !== 'POST') {
    // Return a "method not allowed" error
    return res.status(405).end();
  }
  // TODO(developer) check that the user is authorized to upload

  // Get a reference to the destination file in GCS
  const file = storage.bucket(req.body.bucket).file(req.body.filename);

  // Create a temporary upload URL
  const expiresAtMs = Date.now() + 300000; // Link expires in 5 minutes
  const config = {
    action: 'write',
    expires: expiresAtMs,
    contentType: req.body.contentType,
  };

  file.getSignedUrl(config, (err, url) => {
    if (err) {
      console.error(err);
      res.status(500).end();
      return;
    }
    res.send(url);
  });
};



create a copy of study sauce template?


In [ ]:
var uuid = require('uuid/v1');
var importer = require('../Core');
var getSheet = importer.import('get sheet identifier');
var copyFile = importer.import('copy a file on google drive');
var listDrive = importer.import('list google drive files');
var insertPermission = importer.import('insert google drive permissions');

function copyStudy(email) {
    var fileId;
    return listDrive()
        .then(r => fileId = r.filter(f => f.name === 'Study sauce template')[0].id)
        .then(() => copyFile(fileId, 'Study sauce ' + uuid().substr(0, 5)))
        .then(id => insertPermission(id, email))
        .then(() => fileId)
}

async function handler(req, res) {
    var file;
    var email = (req.body || {})['email'] || (req.form || {})['email'] || (req.query || {})['email'];
    res.set('Access-Control-Allow-Origin', '*');
    return await copyStudy(email)
        // add sheet to signup script with email
        .then(fileId => file = fileId)
        .then(() => getSheet(file, email))
        .then(r => res.status(200).send(file))
        .catch(e => res.status(500).send(Object.getOwnPropertyNames(e).reduce((alt, key) => {
            alt[key] = e[key] + '';
            return alt;
        }, {})));
}

module.exports = {
    copyStudy,
    handler
}


render study sauce cards page?


In [ ]:
var uuid = require('uuid/v1');
var getTemplates = importer.import('templates google sheet');
var wrapTemplate = importer.import('output google sheet template');
var getTemplateProperties = importer.import('google sheet template properties');
var collectTemplateResources = importer.import('collect google sheet resources');
var streamToGoogle = importer.import('upload files google cloud');

var config = 'https://docs.google.com/spreadsheets/d/1EGwxT6InTXuvpLujnwKV3asEFDZhhZk2LdosjW2Tz_M/edit#gid=851799338'
var bucket = 'studysauce.sheet-to-web.com';

function createStream(newPage) {
    var stream = new Readable();
    stream.push(newPage);
    stream.push(null);
    return Promise.resolve(stream);
}

function renderCards(cards) {
    // TODO: create a document and render cards from that
    var docId = config.replace(/https:\/\/docs.google.com\/spreadsheets\/d\/|\/edit.*$|\/copy.*$/ig, '');
    var name = uuid().substr(0, 5) + (new Date()).getTime();
    var properties = {
        'cards-cards-link': '/cards/' + name
    }, templates, key;
    
    return getTemplates(docId)
        .then(t => {
            templates = t;
            templates['cards-cards'] = {template: {rows: [[`{{> ${key}/${filler}-link}}`]]}}
            return getTemplateProperties('cards-cards', Object.assign({}, properties, {
                'cards-data': cards || [{
                    cards: '/cards/' + name,
                    type: 'Flash card',
                    prompt: 'What is the greek word for write/draw?',
                    answer: 'Graph',
                    'possible-1': 'Graph'
                }]
            }), templates);
        })
        .then(() => wrapTemplate('cards/' + name, properties[key], properties))
        .then(page => createStream(page))
        .then(stream => streamToGoogle('cards/' + name + '.html', bucket, stream, {
            contentType: 'text/html; charset=utf-8'
        }))
        .then(resources => {
            console.log(resources);
            return '/cards/' + name + '.html';
        })
}

async function handler(req, res) {
    var file;
    var cards = (req.body || {})['cards'] || (req.form || {})['cards'] || (req.query || {})['cards'];
    res.set('Access-Control-Allow-Origin', '*');
    return await renderCards(cards)
        // add sheet to signup script with email
        .then(r => res.status(200).send(r))
        .catch(e => res.status(500).send(Object.getOwnPropertyNames(e).reduce((alt, key) => {
            alt[key] = e[key] + '';
            return alt;
        }, {})));
}

module.exports = {
    renderCards,
    handler
}


package.json?

In [ ]:
{
    "name": "SheetToWeb",
    "description": "Marketing site functions",
    "license": "UNLICENSED",
    "scripts": {
    },
    "engines": {
        "node": ">= 8",
        "npm": ">= 4"
    },
    "repository": {
        "type": "git",
        "url": "git+https://github.com/megamindbrian/jupytangular.git"
    },
    "dependencies": {
        "@google-cloud/compute": "^0.12.0",
        "@google-cloud/storage": "^2.5.0",
        "googleapis": "^39.2.0",
        "jsdom": "^14.0.0",
        "mustache": "^3.0.1",
        "remarkable": "^1.7.1"
    }
}
